Step 2. Rread the M-scripts. Create a notebook out of them.

http://nbviewer.jupyter.org/github/maxalbert/auto-exec-notebook/tree/master/

In [106]:
import os 
try:
    from os import scandir, walk
except ImportError:
    from scandir import scandir, walk
from nbformat import v4 as nbf
import re

In [107]:
def get_m_files(d="."):
    exts=["m"]
    for root, dirs, files in walk(d, topdown=False):
        for name in files:
            if os.path.splitext(name)[1].strip('.') in exts:
                yield os.path.abspath(os.path.join(root, name))

In [108]:
for m in get_m_files():
    ipynb = os.path.splitext(m)[0]+".ipynb"
    break
example, section = os.path.splitext(os.path.basename(m))[0].split("_")
title = "{} - {}".format(re.sub(r"(\w)([A-Z])", r"\1 \2", example),
                         re.sub(r"(\w)([A-Z])", r"\1 \2", section))
slug = "{}-{}".format(re.sub(r"(\w)([A-Z])", r"\1_\2", example),
                         re.sub(r"(\w)([A-Z])", r"\1_\2", section))

In [109]:
with open (m, "r") as fid:
    lines=fid.readlines()

In [239]:
lines

['%% Aircraft Pitch: Digital Controller Design\n',
 '%\n',
 '% Key MATLAB commands used in this tutorial are:\n',
 '% <http://www.mathworks.com/help/toolbox/control/ref/ss.html |ss|> , \n',
 '% <http://www.mathworks.com/help/toolbox/control/ref/c2d.html |c2d|> , \n',
 '% <http://www.mathworks.com/help/techdoc/ref/rank.html |rank|> ,\n',
 '% <http://www.mathworks.com/help/toolbox/control/ref/ctrb.html |ctrb|> , \n',
 '% <http://www.mathworks.com/help/toolbox/control/ref/dlqr.html |dlqr|> , \n',
 '% <http://www.mathworks.com/help/toolbox/control/ref/lsim.html |lsim|> ,\n',
 '% <http://www.mathworks.com/help/techdoc/ref/stairs.html |stairs|> \n',
 '% \n',
 '%%\n',
 '% In this page, we will consider the digital version of the aircraft pitch \n',
 '% control problem. A sampled-data version of the airplance pitch dynamics can\n',
 '% be obtained from the continuous model, as we will describe. In\n',
 '% this example we will apply state-space techniques to design the controller.  \n',
 '%\n',

In [252]:
nb = nbf.new_notebook()
body=None

regex="""<<([^.]+)\.([\w]+)>>"""
R=re.compile(regex)

notebook_cells=list()

class matlab_to_python_line_transmogrification(object):
    HEADER=0
    COMMENT=1
    CODE=2
    MARKDOWN=3
    """ trans·mog·ri·fied
        trans·mog·ri·fy·ing
        transitive verb
        to change or alter greatly and often with grotesque or humorous effect
    """
    def __init__(self, line):
        self.line = line
        # print("B. "+line)
        if line.startswith("%%"):
            self.type=matlab_to_python_line_transmogrification.HEADER
            self.cell_type = matlab_to_python_line_transmogrification.MARKDOWN
        elif line.startswith("%"):
            self.type=matlab_to_python_line_transmogrification.COMMENT
            self.cell_type = matlab_to_python_line_transmogrification.MARKDOWN
        else:
            self.type=matlab_to_python_line_transmogrification.CODE
            self.cell_type = matlab_to_python_line_transmogrification.CODE
            
    def __repr__(self):
        return "mtplt({})".format(self.line)

    def __str__(self):
        line=self.line.strip()
        line=line.strip("% ")
        if len(line)==0:
            return ""
        if self.type == matlab_to_python_line_transmogrification.HEADER:    
            return "\n\n# {}\n\n".format(line)
        if self.type == matlab_to_python_line_transmogrification.COMMENT:          
            badwords=["Key MATLAB commands", "www.mathworks.com"]
            for badword in badwords:
                if badword in line:
                    return ""
            if line.startswith("* "):
                line="\n"+line
            prefix=""
            postfix=""
            return "{}{}{}".format(prefix,line,postfix)
        if self.type == matlab_to_python_line_transmogrification.CODE:
            line=self.line
            # print("C. "+line)
            return "{}".format(line)
        return ""
last_mtplt=None
for line in lines:
    #print("A. "+line)
    mtplt = matlab_to_python_line_transmogrification(line)
    # print(mtplt.cell_type)
    #print("'{}' -> '{}'".format(line,mtplt))
    if last_mtplt is None:
        running_block=""
    else:
        if mtplt.cell_type != last_mtplt.cell_type:
            if last_mtplt.cell_type == 3:
                cell_tmp = nbf.new_markdown_cell(running_block.strip())
            if last_mtplt.cell_type == 2:
                #print(running_block)
                cell_tmp = nbf.new_code_cell(running_block.strip())
            notebook_cells.append(cell_tmp)
            running_block=str_tmp=str(mtplt)
            last_mtplt = mtplt
            continue
    str_tmp=str(mtplt)
    if len(str_tmp) == 0:
        continue
    if running_block.endswith("\n"):
        running_block=(running_block+str_tmp)
    else:
        running_block=(running_block+" "+str_tmp)
    last_mtplt = mtplt

def cell_cleaner_upperer(notebook_cell):
    source = notebook_cell["source"]
    if notebook_cell["cell_type"] is "markdown":
        # Non inline latex
        source=re.sub(r"\$\$([^$]+)\$\$",r"\n\n$$\n\1\n$$\n\n",source)
        # Embedded Functions
        source=re.sub(r"\|([\w_]+)\|",r"```\1```",source)
        # Links to other
        source=re.sub(r"< \?example=([\w]+)&section=([\w]+) ([^>]+)>",
                      r"[\3](/examples/\1/\1-\2.ipynb)",source)
    if notebook_cell["cell_type"] is "code":
        pass
    #print(source, end="")
    notebook_cell["source"] = source
    return notebook_cell

for notebook_cell in notebook_cells:
    cleaned_cell=cell_cleaner_upperer(notebook_cell)
    nb["cells"].append(cleaned_cell)

In [253]:
nikola_metadata=dict()
nikola_metadatas=dict()
nikola_metadata["author"]=""
nikola_metadata["category"]=""
nikola_metadata["date"]=""
nikola_metadata["description"]=""
nikola_metadata["link"]=""
nikola_metadata["slug"]=""
nikola_metadata["tags"]=""
nikola_metadata["title"]=""
nikola_metadata["type"]=""
nikola_metadatas["nikola"]=nikola_metadata
nb["metadata"]=nikola_metadatas

fname = 'test.ipynb'
with open(fname, 'w') as f:
    print(nbf.writes(nb), file=f) 

"Eh, close enough. Next."